Connects to http://ffw.mrcmekong.org/historical_rec.htm and scrapes river data.
There are 21 stations, and flood season data from 2008 to 2016, and dry season data from 2013-2017. Cause the MRC won't return my email asking for data.
Lik Khian

In [2]:
import numpy as np
from bs4 import BeautifulSoup
import urllib.request as ulib

In [6]:
def check(value):
    try:
        return float(value)
    except:
        return float('nan')

def tablize(link):
    r=ulib.urlopen(link).read()
    soup = BeautifulSoup(r, "lxml")
    riverLevel=soup.find_all(id='table6')
    return riverLevel

def tablize_rainfall(link):
    r=ulib.urlopen(link).read()
    soup = BeautifulSoup(r, "lxml")
    rainfall=soup.find_all(id='table7')
    return rainfall
    
def scrapeNcollateWet(fileName,link,yr):
    #raw=tablize_rainfall(link) #for rainfall
    raw=tablize(link) #for river height
    jun=[]
    jul=[]
    aug=[]
    sep=[]
    octo=[]
    for row in raw[0].find_all('tr')[2:]:
        col = row.find_all('td')
        day = float(col[0].string.strip())
        if (day<=30):
            jun.append(check(col[1].get_text()))
            sep.append(check(col[4].get_text()))
        if (day<=31):
            jul.append(check(col[2].get_text()))
            aug.append(check(col[3].get_text()))
            octo.append(check(col[5].get_text()))
    streamHeight=jun+jul+aug+sep+octo
    print (fileName,yr,len(streamHeight),np.isnan(streamHeight).sum())
    f=open('../HydrologySection/river_analytics/MRC_alldata/'+fileName+'_JJASO_'+yr+'.txt','w')
    #f=open('../HydrologySection/river_analytics/MRC_rainfall/'+fileName+'_JJASO_'+yr+'.txt','w')
    for ii in streamHeight:
        f.write('%.2f\n' %ii)
    f.close()

def scrapeNcollateDry(fileName,link,yr):
    print(fileName,yr)
    raw=tablize_rainfall(link) #for rainfall
    #raw=tablize(link) #for river height
    nov=[]
    dec=[]
    jan=[]
    feb=[]
    mar=[]
    apr=[]
    may=[]
    for row in raw[0].find_all('tr')[2:]:
        col = row.find_all('td')
        day = float(col[0].string.strip())
        if (day<=28):
            feb.append(check(col[4].get_text()))
        if (day==29 and not bool(int(yr[-4:])%4)):
            feb.append(check(col[4].get_text()))
        if (day<=30):
            nov.append(check(col[1].get_text()))
            apr.append(check(col[6].get_text()))
        if (day<=31):
            dec.append(check(col[2].get_text()))
            jan.append(check(col[3].get_text()))
            mar.append(check(col[5].get_text()))
            may.append(check(col[7].get_text()))
    streamHeight=nov+dec+jan+feb+mar+apr+may
    print (fileName,yr,len(streamHeight),np.isnan(streamHeight).sum())
    f=open('../HydrologySection/river_analytics/MRC_alldata/'+fileName+'_NDJFMAM_'+yr+'.txt','w')
    for ii in streamHeight:
        f.write('%.2f\n' %ii)
    f.close()

In [7]:
sites={'chiangsaen':'csa','luangprabang':'lua','chiangkhan':'ckh','vientiane':'vie',\
       'nongkhai':'non','paksane':'pak','nakhonphanom':'nak','mukdahan':'muk',\
       'pakse':'pks','stungtreng':'str','kratie':'kra',\
       'kompongcham':'kom','phnompenhbassac':'ppb',\
       'kohkhel':'koh','neakluong':'nea','prekkdam':'pre','tanchau':'tch','chaudoc':'cdo'}
#,'phnompenhport':'ppp','thakhek':'tha','savannakhet':'sav', only in wet season, not in dry.
year=np.arange(2008,2017).astype(str)
yeardry=['2013_2014','2014_2015','2015_2016','2016_2017']

for yr in year:
    for key in sites:
        scrapeNcollateWet(key,'http://ffw.mrcmekong.org/historical_data/'+yr+'/stations_historical/historical_'+sites[key]+'.htm',yr)
for yrd in yeardry:
    for key in sites:
        scrapeNcollateDry(key,'http://ffw.mrcmekong.org/historical_data_dry/'+yrd+'/stations_dry/historical_dry_'+sites[key]+'.htm',yrd)




kratie 2013_2014


IndexError: list index out of range

In [104]:
yr="2015_2016"
day=29
print(bool(int(yr[-4:])%4))
print(~bool(int(yr[-4:])%4))
print(day<=28)

print(day==29 and not bool(int(yr[-4:])%4))

False
-1
False
True
